In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


# JsonOutputParser

1. 출력 형식 정의: 
    - AI 모델의 텍스트 응답을 JSON 형태로 파싱 
    - 구조화된 데이터를 생성하고 관리하는데 적합
2. 유연성: 
    - JSON 데이터의 키-값 구조를 간단히 처리 가능 
3. 사용 용이성:
    - Pydantic 같은 추가 검증 없이 간단히 JSON 형식으로 파싱하는데 적합

[Reference] https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html

In [3]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import JsonOutputParser

In [4]:
# 답변 스키마 정의 
class JsonOutputClass(BaseModel):
    description: str = Field(description='주제에 대한 명확하고 간결한 설명')
    hashtags: str = Field(description='해시태그 형식의 키워드 (3개)')

In [5]:
# 질의 내용 
question = '미세먼지의 심각성에 대해 정리해 주세요.'

# 파서 정의 
output_parser = JsonOutputParser(pydantic_object=JsonOutputClass)
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 명확하고 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드 (3개)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [6]:
# 템플릿 정의 
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assistant. Please answer the following questions.'),
        ('user','#Format: {format_instructions}\n\n#Question: {question}'),
    ]
)

# 템플릿에 파서 적용 
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

# 모델 정의 
model = ChatOpenAI(temperature=0)

# 체인 정의 
chain = prompt | model | output_parser

# 실행 
answer = chain.invoke({'question':question})

In [8]:
print(answer)

{'description': '미세먼지는 대기 중에 미세한 입자로, 주로 공장, 자동차, 난방 등으로 인해 대기 중에 발생합니다. 이 입자들은 호흡기로 들어가면 폐에 침착되어 호흡기 질환을 유발할 수 있으며, 눈이 따가워지고 피부가 가렵거나 건조해지는 피부 질환을 유발할 수도 있습니다. 미세먼지 농도가 높을수록 심각한 건강 문제를 유발할 수 있으므로 주의가 필요합니다.', 'hashtags': '미세먼지, 환경, 건강'}


In [9]:
type(answer)

dict

(참고) Pydantic 없이 사용
- `JsonOutputParser`는 Pydantic 없이도 사용 가능 
- 선언하지 않을 경우, 스키마가 어떻게 되는지 정보 확인 안됨

In [12]:
# 질의 내용 
# question = '미세먼지의 심각성에 대해 정리해 주세요.'
question = '미세먼지의 심각성에 대해 정리해 주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요.'

# 파서 정의 
# output_parser = JsonOutputParser(pydantic_object=JsonOutputClass)
output_parser = JsonOutputParser()

# 템플릿 정의 
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assistant. Please answer the following questions.'),
        ('user','#Format: {format_instructions}\n\n#Question: {question}'),
    ]
)

# 템플릿에 파서 적용 
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

# 모델 정의 
model = ChatOpenAI(temperature=0)

# 체인 정의 
chain = prompt | model | output_parser

# 실행 
answer = chain.invoke({'question':question})

In [13]:
print(answer)

{'description': '미세먼지는 대기 중에 미세한 입자로 인체에 해로운 영향을 미치는 대기오염물질입니다. 주로 공장, 자동차, 난방 등으로 인해 발생하며 호흡기 질환, 알레르기 증상, 심장 질환 등을 유발할 수 있습니다.', 'hashtags': ['미세먼지', '대기오염', '호흡기질환', '환경오염']}


In [14]:
type(answer)

dict

In [15]:
print(output_parser.get_format_instructions())

Return a JSON object.


-----
** End of Documents **